In [2]:
import h5py
%matplotlib inline
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import sys
import shutil
import glob
import optparse
import os
import json
import pandas as pd
import numpy as np
import pylab as pl
import scipy.stats as stats
import seaborn as sns
sns.set_style("ticks")
sns.set()
sns.set_color_codes()

def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

In [4]:
aggregate_root = '/n/coxfs01/cechavarria/2p-aggregate/scenes'

# sess_list = ['JC097_20190621','JC085_20190624','JC097_20190628','JC097_20190704','JC085_20190712']
# area = 'V1'

sess_list = ['JC080_20190619','JC091_20190621','JC091_20190628','JC097_20190702','JC091_20190703','JC085_20190704','JC097_20190708']
area = 'LM'

# sess_list = ['JC091_20190625','JC091_20190701','JC091_20190705']
# area = 'LI'

num_animals = 2
num_session = 3

response_type = 'norm_df'

filter_crit = 'zscore'
filter_thresh = 1

#define paths
aggregate_file_dir = os.path.join(aggregate_root,area,'files','filtered_responses')
fig_out_dir = os.path.join(aggregate_root,area,'figures')

if not os.path.isdir(fig_out_dir):
        os.makedirs(fig_out_dir)

In [28]:
bar_loc = np.zeros((3,))
width = 0.4         # the width of the bars
xloc = 1
count = 0

for j in range(3):
    bar_loc[count] = xloc
    xloc = xloc + width
    count = count+1


In [29]:


fig,axes = plt.subplots(num_animals,num_session,figsize=(20,20))

if num_animals <2:
    axes = np.expand_dims(axes,0)

animalids = []
sess_count = []


for sess_idx, sess in enumerate(sess_list):
#sess_idx = 0
#sess = sess_list[sess_idx]
    i1 = findOccurrences(sess,'_')[0]
    animalid = sess[0:i1]
    session = sess[i1+1:]

    if animalid not in animalids:
        animalids.append(animalid)
        sess_count.append(0)

    sess_count[animalids.index(animalid)] = sess_count[animalids.index(animalid)]+1

    data_array_fn = '%s_filtered_%s_responses_thresh_%s_%i.hdf5'%(sess, response_type, filter_crit, filter_thresh)
    data_array_filepath = os.path.join(aggregate_file_dir, data_array_fn)
    data_grp = h5py.File(data_array_filepath, 'r')



    curr_slice = 'Slice01'#hard-code planar data for now

    num_active_rois = int(np.array(data_grp['/'.join([curr_slice,'n_active_rois'])]))
    frac_active_rois = float(np.array(data_grp['/'.join([curr_slice,'frac_active_rois'])]))

    neuron_count = num_active_rois
    neuron_frac = frac_active_rois

    sess_response = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cond_mean_across_neurons'])])
    response_se = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cond_se_across_neurons'])])


    
    ax = axes[animalids.index(animalid),sess_count[animalids.index(animalid)]-1]
    ax.bar(bar_loc[0],sess_response[0],width,color = 'b', alpha = 1, yerr = response_se[0])
    ax.bar(bar_loc[1],sess_response[2],width,color = 'r', alpha = 1, yerr = response_se[2])
    ax.bar(bar_loc[2],sess_response[1],width,color = 'g', alpha = 1, yerr = response_se[1])
    
    
    ymin,ymax = ax.get_ylim()
    xmin,xmax = ax.get_xlim()
    ax.set_xticks([],[])
    ax.set_xlabel('Condition',fontsize = 15)
    ax.set_ylabel('Average Response',fontsize = 15)
    ax.text(bar_loc[0]-.25, ymax, 'n=%i, f=%.04f' % (neuron_count, neuron_frac), fontsize=10)
    ax.set_title('%s | %i'%(animalid, sess_count[animalids.index(animalid)]))
    
    data_grp.close()
fig_fn = '%s_avg_response_per_cond_%s_thresh_%s_%i.png'%(area,response_type,filter_crit,filter_thresh)
fig_file_path = os.path.join(fig_out_dir, fig_fn)
plt.savefig(fig_file_path)

plt.close()

In [17]:
animalids = []
sess_count = []

#put things into pandas df for plotting
resp_dfs = []

animalids = []
sess_count = []
neuron_count = np.empty((10,10))
neuron_frac = np.empty((10,10))
resp_mean = np.ones((3,10,10))*np.nan
resp_se = np.ones((3,10,10))*np.nan

ylabel = 'Average Response'

for sess_idx, sess in enumerate(sess_list):
    print(sess)
    #sess_idx = 0
    #sess = sess_list[sess_idx]
    i1 = findOccurrences(sess,'_')[0]
    animalid = sess[0:i1]
    session = sess[i1+1:]


    if animalid not in animalids:
        animalids.append(animalid)
        sess_count.append(0)

    animal_idx = animalids.index(animalid)
    sess_count[animal_idx] = sess_count[animal_idx]+1

    data_array_fn = '%s_filtered_%s_responses_thresh_%s_%i.hdf5'%(sess, response_type, filter_crit, filter_thresh)
    data_array_filepath = os.path.join(aggregate_file_dir, data_array_fn)
    data_grp = h5py.File(data_array_filepath, 'r')


    curr_slice = 'Slice01'#hard-code planar data for now

    num_active_rois= int(np.array(data_grp['/'.join([curr_slice,'n_active_rois'])]))
    frac_active_rois = float(np.array(data_grp['/'.join([curr_slice,'frac_active_rois'])]))

    neuron_count[animal_idx,sess_count[animal_idx]-1] = num_active_rois
    neuron_frac[animal_idx,sess_count[animal_idx]-1]  = frac_active_rois
    active_cell_idx = np.array(data_grp['/'.join([curr_slice, 'active_cell_idx'])])

    resp_mean[:,animal_idx,sess_count[animal_idx]-1] = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cond_mean_across_neurons'])])
    resp_se[:,animal_idx,sess_count[animal_idx]-1] = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cond_se_across_neurons'])])


    sess_response = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cond_per_neuron'])])

    sess_response = sess_response[[0,2,1],:]
    resp_mean = resp_mean[[0,2,1],:,:]
    resp_se = resp_mean[[0,2,1],:,:]
    for cidx in range(sess_response.shape[0]):
        response = sess_response[cidx,active_cell_idx]
        cell = np.arange(num_active_rois)
        cond = np.ones((num_active_rois,))*cidx
        mdf = pd.DataFrame({'%s' % ylabel: response,
                            'animalid': animalid,
                            'session': sess_count[animalids.index(animalid)],
                            'response': response,
                            'cell': cell,
                            'cond': cond,
                           })

        resp_dfs.append(mdf)
resp_dfs = pd.concat(resp_dfs, axis=0)



JC097_20190621
JC085_20190624
JC097_20190628
JC097_20190704
JC085_20190712


In [18]:
bar_loc = np.zeros((3,))
width = 0.4         # the width of the bars
xloc = 0
count = 0
gap = 0.6
for j in range(3):
    bar_loc[count] = xloc
    xloc = xloc + width+gap
    count = count+1

palette = sns.color_palette(["#4c72b0","#c44e52","#55a868",])
sns.set_palette(palette)

p = sns.catplot(row = 'animalid',col = 'session', x='cond', y='response', kind="swarm", hue = 'cond',data=resp_dfs,size = 10);
pl.subplots_adjust(wspace=0.1, hspace=0.2, top=0.85, bottom=0.1, left=0.1)

axes = p.axes
ymin,ymax = axes[0][0].get_ylim()
xmin,xmax = axes[0][0].get_xlim()
start_val = 0.0
end_val = 1.0 #hard-coding
for ri in range(p.axes.shape[0]):
    for ci in range(p.axes.shape[1]):
        #print ri, ci
         
        p.axes[ri,ci].text(-0.25, ymax+(ymax*0), 'n=%i, f = %.04f' % (neuron_count[ri,ci],neuron_frac[ri,ci]), fontsize=10)
        p.axes[ri,ci].set_xticks([],[])
        p.axes[ri,ci].axhline(y=0, xmin=xmin, xmax= xmax, linewidth=1, color='k',linestyle = '--')

        for idx in range(3):
            p.axes[ri,ci].hlines(y = resp_mean[idx,ri,ci], xmin=bar_loc[idx]-(width/2), xmax = bar_loc[idx]+(width/2), linewidth=2, color='k',linestyle = '-')
            p.axes[ri,ci].hlines(y = resp_mean[idx,ri,ci] + resp_se[idx,ri,ci], xmin=bar_loc[idx]-(width/2), xmax = bar_loc[idx]+(width/2), linewidth=1, color='k',linestyle = '--')
            p.axes[ri,ci].hlines(y = resp_mean[idx,ri,ci] - resp_se[idx,ri,ci], xmin=bar_loc[idx]-(width/2), xmax = bar_loc[idx]+(width/2), linewidth=1, color='k',linestyle = '--')




fig_fn = '%s_response_per_cond_per_neuron_%s_thresh_%s_%i.png'%(area,response_type,filter_crit,filter_thresh)
fig_file_path = os.path.join(fig_out_dir, fig_fn)

p.savefig(fig_file_path)
plt.close()

/home/cesar/envs/suite2p/lib/python3.6/site-packages/seaborn/categorical.py:3692: UserWarning: The `size` paramter has been renamed to `height`; please update your code.
  warnings.warn(msg, UserWarning)


In [30]:
bar_loc = np.zeros((60,))
width = 0.4         # the width of the bars
gap = .5
xloc = 1
count = 0
for i in range(20):
    for j in range(3):
        bar_loc[count] = xloc
        xloc = xloc + width
        count = count+1
    xloc = xloc + gap


In [32]:
fig,axes = plt.subplots(num_animals,num_session,figsize=(80,30))
if num_animals <2:
    axes = np.expand_dims(axes,0)

animalids = []
sess_count = []


for sess_idx, sess in enumerate(sess_list):
#sess_idx = 0
#sess = sess_list[sess_idx]
    i1 = findOccurrences(sess,'_')[0]
    animalid = sess[0:i1]
    session = sess[i1+1:]

    if animalid not in animalids:
        animalids.append(animalid)
        sess_count.append(0)

    sess_count[animalids.index(animalid)] = sess_count[animalids.index(animalid)]+1

    data_array_fn = '%s_filtered_%s_responses_thresh_%s_%i.hdf5'%(sess, response_type, filter_crit, filter_thresh)
    data_array_filepath = os.path.join(aggregate_file_dir, data_array_fn)
    data_grp = h5py.File(data_array_filepath, 'r')



    curr_slice = 'Slice01'#hard-code planar data for now

    active_rois_per_img = np.array(data_grp['/'.join([curr_slice,'active_rois_per_img'])])

    sess_response = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cfg_mean_across_neurons'])])
    response_se = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cfg_se_across_neurons'])])


    
    ax = axes[animalids.index(animalid),sess_count[animalids.index(animalid)]-1]
    ax.bar(bar_loc[0:len(bar_loc):3],sess_response[0:len(bar_loc):3],width,color = 'b', alpha = 1, yerr = response_se[0:len(bar_loc):3])
    ax.bar(bar_loc[1:len(bar_loc):3],sess_response[2:len(bar_loc):3],width,color = 'r', alpha = 1, yerr = response_se[2:len(bar_loc):3])
    ax.bar(bar_loc[2:len(bar_loc):3],sess_response[1:len(bar_loc):3],width,color = 'g', alpha = 1, yerr = response_se[1:len(bar_loc):3])
    
    ymin,ymax = ax.get_ylim()
    xmin,xmax = ax.get_xlim()
    plt.axhline(y=0, xmin=xmin, xmax= xmax, linewidth=1, color='k',linestyle = '-')
    xtick_loc = bar_loc[1:len(bar_loc):3]
    xtick_label = np.unique(np.arange(20)+1).astype('int')

    ax.set_xticks(xtick_loc)
    ax.set_xticklabels(xtick_label.tolist())
    ax.set_xlabel('Image',fontsize = 15)
    ax.set_ylabel('Average Response',fontsize = 15)
    count = 0
    
    for idx in bar_loc[1:len(bar_loc):3]:
        ax.text(idx-.25, ymax-(.02*ymax), 'n=%i' % active_rois_per_img[count], fontsize=10)
        count = count +3
    
    ax.set_title('%s | %i'%(animalid, sess_count[animalids.index(animalid)]))
    
    data_grp.close()
fig_fn = '%s_avg_response_per_config_%s_thresh_%s_%i.png'%(area,response_type,filter_crit,filter_thresh)
fig_file_path = os.path.join(fig_out_dir, fig_fn)
plt.savefig(fig_file_path)

plt.close()

In [33]:
fig,axes = plt.subplots(num_animals,num_session,figsize=(80,30))
if num_animals <2:
    axes = np.expand_dims(axes,0)

animalids = []
sess_count = []


for sess_idx, sess in enumerate(sess_list):
#sess_idx = 0
#sess = sess_list[sess_idx]
    i1 = findOccurrences(sess,'_')[0]
    animalid = sess[0:i1]
    session = sess[i1+1:]

    if animalid not in animalids:
        animalids.append(animalid)
        sess_count.append(0)

    sess_count[animalids.index(animalid)] = sess_count[animalids.index(animalid)]+1

    data_array_fn = '%s_filtered_%s_responses_thresh_%s_%i.hdf5'%(sess, response_type, filter_crit, filter_thresh)
    data_array_filepath = os.path.join(aggregate_file_dir, data_array_fn)
    data_grp = h5py.File(data_array_filepath, 'r')



    curr_slice = 'Slice01'#hard-code planar data for now

    active_rois_per_config = np.array(data_grp['/'.join([curr_slice,'active_rois_per_config'])])


    ax = axes[animalids.index(animalid),sess_count[animalids.index(animalid)]-1]
    ax.bar(bar_loc[0:len(bar_loc):3],active_rois_per_config[0:len(bar_loc):3],width,color = 'b')
    ax.bar(bar_loc[1:len(bar_loc):3],active_rois_per_config[2:len(bar_loc):3],width,color = 'r')
    ax.bar(bar_loc[2:len(bar_loc):3],active_rois_per_config[1:len(bar_loc):3],width,color = 'g')
    
    ymin,ymax = ax.get_ylim()
    xmin,xmax = ax.get_xlim()
    plt.axhline(y=0, xmin=xmin, xmax= xmax, linewidth=1, color='k',linestyle = '-')
    xtick_loc = bar_loc[1:len(bar_loc):3]
    xtick_label = np.unique(np.arange(20)+1).astype('int')

    ax.set_xticks(xtick_loc)
    ax.set_xticklabels(xtick_label.tolist())
    ax.set_xlabel('Image',fontsize = 15)
    ax.set_ylabel('Cell Count',fontsize = 15)
    count = 0
    
    
    ax.set_title('%s | %i'%(animalid, sess_count[animalids.index(animalid)]))
    
    data_grp.close()
fig_fn = '%s_cells_per_config_%s_thresh_%s_%i.png'%(area,response_type,filter_crit,filter_thresh)
fig_file_path = os.path.join(fig_out_dir, fig_fn)
plt.savefig(fig_file_path)

plt.close()

In [1]:
#modulation index figures
title_list = ['Corr-Match vs SF-Match','Original vs SF-Match','Original vs Corr-Match']
for cidx in range(len(title_list)):
#cidx = 0 
#modulation index figure

    fig,axes = plt.subplots(num_animals,num_session,figsize=(45,15))
    if num_animals <2:
        axes = np.expand_dims(axes,0)

    animalids = []
    sess_count = []


    for sess_idx, sess in enumerate(sess_list):
    #sess_idx = 0
    #sess = sess_list[sess_idx]
        i1 = findOccurrences(sess,'_')[0]
        animalid = sess[0:i1]
        session = sess[i1+1:]


        if animalid not in animalids:
            animalids.append(animalid)
            sess_count.append(0)

        animal_idx = animalids.index(animalid)
        sess_count[animal_idx] = sess_count[animal_idx]+1

        data_array_fn = '%s_filtered_%s_responses_thresh_%s_%i.hdf5'%(sess, response_type, filter_crit, filter_thresh)
        data_array_filepath = os.path.join(aggregate_file_dir, data_array_fn)
        data_grp = h5py.File(data_array_filepath, 'r')


        curr_slice = 'Slice01'#hard-code planar data for now

        active_rois_per_img = np.array(data_grp['/'.join([curr_slice,'active_rois_per_img'])])

        mod_idx_mean = np.array(data_grp['/'.join([curr_slice, 'mod_idx_mean_across_neurons'])])
        mod_idx_se = np.array(data_grp['/'.join([curr_slice, 'mod_idx_se_across_neurons'])])

        ax = axes[animal_idx,sess_count[animal_idx]-1]
        ax.bar(np.arange(20),mod_idx_mean[cidx:60:3],yerr = mod_idx_se[cidx:60:3])
        #plot unity
        xmin, xmax = ax.get_xlim()
        ymin, ymax = ax.get_ylim()
        if ymin<0:
            ylimit = np.max(np.abs(np.array([ymin,ymax])))
            ymin = -ylimit
            ymax = ylimit
            ax.set_ylim([ymin,ymax])

        ax.axhline(y=0, xmin=xmin, xmax= xmax, linewidth=1, color='k',linestyle = '-')
        for idx in range(20):
            ax.text(idx-.25, ymax, 'n=%i' % active_rois_per_img[idx*3], fontsize=10)

        ax.set_xticks(np.arange(20))
        xtick_label = np.arange(20)+1
        ax.set_xticklabels(xtick_label.tolist())
        ax.set_xlabel('Image',fontsize = 15)
        ax.set_ylabel('Modulation Index',fontsize = 15)

        fig.suptitle(title_list[cidx])
        fig_fn = '%s_%s_mod_idx_%s_thresh_%s_%i.pdf'%(area,title_list[cidx],response_type,filter_crit,filter_thresh)
        fig_file_path = os.path.join(fig_out_dir, fig_fn)
        fig.savefig(fig_file_path)
        plt.close()

NameError: name 'plt' is not defined

In [7]:
#pool all neurons


#put things into pandas df for plotting
resp_dfs = []

animalids = []
sess_count = []
active_cell_count = 0
total_cell_count = 0

ylabel = 'Average Response'

for sess_idx, sess in enumerate(sess_list):
# sess_idx = 0
# sess = sess_list[sess_idx]
    i1 = findOccurrences(sess,'_')[0]
    animalid = sess[0:i1]
    session = sess[i1+1:]


    if animalid not in animalids:
        animalids.append(animalid)
        sess_count.append(0)

    animal_idx = animalids.index(animalid)
    sess_count[animal_idx] = sess_count[animal_idx]+1

    data_array_fn = '%s_filtered_%s_responses_thresh_%s_%i.hdf5'%(sess, response_type, filter_crit, filter_thresh)
    data_array_filepath = os.path.join(aggregate_file_dir, data_array_fn)
    data_grp = h5py.File(data_array_filepath, 'r')


    curr_slice = 'Slice01'#hard-code planar data for now


    active_cell_idx = np.array(data_grp['/'.join([curr_slice, 'active_cell_idx'])])
    total_cell_count = total_cell_count + active_cell_idx.shape[0]
    active_cell_count = active_cell_count + np.nansum(active_cell_idx)
    sess_response = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cond_per_neuron'])])
    
    sess_response = sess_response[[0,2,1],:]
    if sess_idx == 0:
        response_array = sess_response[:,active_cell_idx]
    else:
        response_array = np.hstack((response_array,sess_response[:,active_cell_idx]))


In [8]:
frac_active_cells = active_cell_count/float(total_cell_count)


response_per_cond_mean = np.nanmean(response_array,1)
response_per_cond_se = np.nanstd(response_array,1)/np.sqrt(active_cell_count)

bar_loc = np.zeros((3,))
width = 0.4         # the width of the bars
xloc = 1
count = 0

for j in range(3):
    bar_loc[count] = xloc
    xloc = xloc + width
    count = count+1

fig = plt.figure(figsize=(8,8))
plt.bar(bar_loc[0],response_per_cond_mean[0],width,color = 'b',yerr = response_per_cond_se[0])
plt.bar(bar_loc[1],response_per_cond_mean[1],width,color = 'r',yerr = response_per_cond_se[1])
plt.bar(bar_loc[2],response_per_cond_mean[2],width,color = 'g',yerr = response_per_cond_se[2])

axes = plt.gca()
xmin, xmax = axes.get_xlim()
ymin, ymax = axes.get_ylim()
plt.axhline(y=0, xmin=xmin, xmax= xmax, linewidth=1, color='k',linestyle = '-')

xtick_loc = []
xtick_label = []

plt.xticks(xtick_loc,xtick_label)
plt.xlabel('Condition',fontsize = 15)
plt.ylabel('Average Response',fontsize = 15)
plt.suptitle('Average %s Across Neurons'%(response_type),fontsize = 15)


plt.text(bar_loc[0]-.25, ymax, 'n=%i, f=%.04f' % (active_cell_count, frac_active_cells), fontsize=10)


fig_fn = '%s_avg_response_per_cond_across_neurons_%s_thresh_%s_%i.png'%(area,response_type,filter_crit,filter_thresh)

fig_file_path = os.path.join(fig_out_dir, fig_fn)
plt.savefig(fig_file_path)
plt.close()

In [38]:

ylabel = 'Average Response'

resp_dfs = []
for cidx in range(response_array.shape[0]):
    response = response_array[cidx,:]
    cell = np.arange(active_cell_count)
    cond = np.ones((active_cell_count,))*cidx
    mdf = pd.DataFrame({'%s' % ylabel: response,
                        'response': response,
                        'cell': cell,
                        'cond': cond,
                       })

    resp_dfs.append(mdf)
resp_dfs = pd.concat(resp_dfs, axis=0)

bar_loc = np.arange(0,3)
width = 0.5

palette = sns.color_palette(["#4c72b0","#c44e52","#55a868"])
sns.set_palette(palette)

p = sns.catplot(x='cond', y='response', kind="swarm", hue = 'cond',data=resp_dfs,size = 10);

axes = p.ax
ymin,ymax = axes.get_ylim()
xmin,xmax = axes.get_xlim()

for idx in range(3):
    p.ax.hlines(y = response_per_cond_mean[idx], xmin=bar_loc[idx]-(width/2), xmax = bar_loc[idx]+(width/2), linewidth=2, color='k',linestyle = '-')
    p.ax.hlines(y = response_per_cond_mean[idx] + response_per_cond_se[idx], xmin=bar_loc[idx]-(width/2), xmax = bar_loc[idx]+(width/2), linewidth=1, color='k',linestyle = '--')
    p.ax.hlines(y = response_per_cond_mean[idx] - response_per_cond_se[idx], xmin=bar_loc[idx]-(width/2), xmax = bar_loc[idx]+(width/2), linewidth=1, color='k',linestyle = '--')




p.ax.set_xticks(())
p.ax.set_xlabel('Condition',fontsize = 15)
p.ax.set_ylabel('Average Response',fontsize = 15)
p.fig.suptitle('Average %s Across Neurons'%(response_type),fontsize = 15)


p.ax.text(bar_loc[0]-.25, ymax, 'n=%i, f=%.04f' % (active_cell_count, frac_active_cells), fontsize=10)


fig_fn = '%s_avg_response_per_cond_scatter_neurons_%s_thresh_%s_%i.png'%(area,response_type,filter_crit,filter_thresh)

fig_file_path = os.path.join(fig_out_dir, fig_fn)
plt.savefig(fig_file_path)

plt.close()


/home/cesar/envs/suite2p/lib/python3.6/site-packages/seaborn/categorical.py:3692: UserWarning: The `size` paramter has been renamed to `height`; please update your code.
  warnings.warn(msg, UserWarning)


In [6]:
h

NameError: name 'fig_file_path' is not defined

In [ ]:
data_grp.close()

In [ ]:
# 

In [ ]:
#----Junk Code below here------